In [16]:
import os
import io
import mimetypes
from zipfile import ZipFile
import re
from datetime import datetime


class Message:
    def __init__(self, first_name, last_name, nickname, dt, text):
        self.first_name = first_name
        self.last_name = last_name
        self.nickname = nickname
        self.dt = datetime.strptime(dt, '%H:%M:%S %d/%m/%Y')
        self.year = self.dt.year
        self.month = self.dt.month
        self.day = self.dt.day
        self.hour = self.dt.hour
        self.minute = self.dt.minute
        self.second = self.dt.second
        self.day_of_week = self.dt.weekday()
        self.__parse_message(text)

        

    def __parse_message(self):
        # TODO: Тип сообщения (приглашение/ответ/пересланное/обычное/что-то еще)
        # TODO: Содержит ли сообщение фотку
        # TODO: Содержит ли сообщение ссылку
        # TODO: придумать еще какие-нибудь атрибуты для сообщений
        pass


    def as_dict(self):
        return {'first_name': self.first_name, 
                'last_name': self.last_name, 
                'nickname': self.nickname,
                'year': self.dt.year,
                'month': self.dt.month,
                'day': self.dt.day,
                'hour': self.dt.hour,
                'minute': self.dt.minute,
                'text': self.text}

    def __str__(self):
        return f"{self.first_name} {self.last_name} wrote {self.text} at {self.dt.strftime('%d-%m-%Y %H:%M:%S')}"

    def __repr__(self):
        return f"{self.first_name} {self.last_name}'s message at {self.dt.strftime('%d-%m-%Y %H:%M:%S')}"


class Reader:
    def __init__(self, path_to_data):
        

        self.path_to_data = path_to_data
        self.messages = []
        self.__parse_archive()
        

    def __check_file(self):
        # TODO: Сделать проверку на архив или текстовый файл
        # TODO: Сделать проверку на неправильный файл
        pass


    def __parse_archive(self):
        with ZipFile(self.path_to_data, 'r') as zipObj:
            for elem in zipObj.namelist():
                with io.TextIOWrapper(zipObj.open(elem), encoding="utf-8") as f:
                    self.__split_file_by_msg(f)

    def __split_file_by_msg(self, fd):
        text = []
        first = True
        for line in fd.readlines():
            if re.fullmatch(r'^\S.+\[\w+\]\s\(\d{2}:\d{2}:\d{2}\s+\d{2}/\d{2}/\d{4}\):\n$', line):
                if not first:
                    msg = Message(first_name, last_name, nickname, dt, '\n'.join(text))
                    self.messages.append(msg)
                    text = []
                else:
                    first = False
                splitted = line.split()
                if len(splitted) == 5:
                    first_name = splitted[0]
                    last_name = splitted[1]
                elif len(splitted) == 4:
                    first_name = splitted[0]
                    last_name = None
                nickname = splitted[-3][1:-1]
                dt = splitted[-2][1:] + ' ' + splitted[-1][:-2]
                continue
            
            text.append(line)

In [17]:
rdr = Reader('C16-702 тру групп.zip')

In [18]:
import re 
  
def find_url(string): 
    regex = re.compile(
        r'^(?:http|ftp)s?://' # http:// or https://
        r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+(?:[A-Z]{2,6}\.?|[A-Z0-9-]{2,}\.?)|' #domain...
        r'localhost|' #localhost...
        r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})' # ...or ip
        r'(?::\d+)?' # optional port
        r'(?:/?|[/?]\S+)$', re.IGNORECASE)

    url = re.findall(regex, string)       
    return [x[0] for x in url] 

In [22]:
import numpy as np
import pandas as pd

In [23]:
pd.DataFrame(map(lambda x: x.as_dict(), rdr.messages))

,first_name,last_name,nickname,year,month,day,hour,minute,text
0,Денис,Колесников,id185184434,2016,9,15,16,7,\n\nchat_invite_user | Андрей Гультяев [angult...
1,Денис,Колесников,id185184434,2016,9,15,16,7,\n\nchat_invite_user | Владислав Файль [vf__21...
2,Денис,Колесников,id185184434,2016,9,15,16,7,\n\nchat_invite_user | Ерназар Искаков [id1582...
3,Александр,Гридин,alexqrid,2016,9,15,19,19,\n\n\n\n\n
4,Александр,Гридин,alexqrid,2016,9,15,19,19,первая подгруппа по англ советую всем на лингу...
...,...,...,...,...,...,...,...,...,...
49469,Илья,Филатов,ilya_filatovv,2020,6,13,11,41,А физру вот не прогуливал в школе\n\n\n\n\n
49470,Никита,Шубин,id185727944,2020,6,13,11,42,\tСофья Емцева [id77761992] (11:41:23 13/06/2...
49471,Владислав,Файль,vf__21,2020,6,13,11,45,\tНикита Шубин [id185727944] (11:42:22 13/06/...
49472,Артём,Наумов,just_awesome_guy,2020,6,13,12,18,Есть итоговая ведомость с оценками?\n\n\n\n\n
